from IPython.display import Markdown, display

intro_text = """
# 🚀 PyArrow Playground
### by **Swap**

Hey there 👋 — welcome to my personal **PyArrow exploration notebook**!
This is where I’m monkeying around, learning, and building a deeper understanding of how
**Apache Arrow** handles memory, arrays, tables, and how it fits with libraries like
**Pandas**, **Polars**, and **DuckDB**.

---

### 💡 What I've Explored until now
- 🧱 How Arrow stores data in a *columnar, zero-copy* form or way
- 📊 Creation and manipulation of **Arrow Arrays** and **Tables**
- ⚙️ Used `pyarrow.compute` for fast math and filtering
- 🔁 Converted data between **PyArrow ↔ Pandas ↔ Polars**
- ⚡ Benchmarked Arrow’s performance, out of curiosity

---

### 🧰 The Toolkit
| Library | What I’m Using It For |
|----------|----------------------|
| 🐍 **Python 3.13** | The usual suspect |
| 🧱 **PyArrow** | The main attraction |
| 📊 **Polars / Pandas** | Easy DataFrame comparisons |
| 🧮 **DuckDB** | Running SQL magic in-memory |
| ⏱ **time / numpy** | Tiny performance tests |

---

### ✍️ A Few Notes
This isn’t a strict tutorial — more of a “**Swap experiments and learns Arrow**” space.
I’ll be running random tests, trying weird stuff, and breaking things on purpose 😅

If it looks messy at times, there's gotta be some decent insight which you might figure out.
Let’s see what this beast called **PyArrow** can really do 🔥

---

📅 **Started:** October 2025
👨‍💻 **Author:** *Swap*
🏗️ **Notebook Mood:** Curious & de - caffeinated ☕
"""

display(Markdown(intro_text))

In [3]:
# In this tab we shall start with the basics
# Let us first start by importing the PyArrow
import pyarrow as pa
print("Import of pyarrow was successful")

Import of pyarrow was successful


In [4]:
# Now we will create an array with some values
days = pa.array([1,2,3,4,5,6,7], type=pa.int8())
# so the first list tells us which values will be there in our list, and the type tells the type of the vector, we must remember at all cost that this a vector list meaning the list cannot have different data types.
print(f"Here is a quick view of the array: {days}")
# So interestingly if we had created the same array with the help of numpy there would not have been commas between the elements, let me show you that in the next cell

Here is a quick view of the array: [
  1,
  2,
  3,
  4,
  5,
  6,
  7
]


In [5]:
import numpy as np

days_with_numpy_1 = np.arange(7)
days_with_numpy_2 = np.array([1,2,3,4,5,6,7])
print(f"So here is the difference between the prior one and this one: {days_with_numpy_1} vs {days_with_numpy_2} and finally the previous one which we had created with pyarrow: {days}")

# See in numpy there are no commas


So here is the difference between the prior one and this one: [0 1 2 3 4 5 6] vs [1 2 3 4 5 6 7] and finally the previous one which we had created with pyarrow: [
  1,
  2,
  3,
  4,
  5,
  6,
  7
]


##### `PyArrow`  arrays are not  “Pythonic”  containers —
##### they’re `immutable`, `low-level`, `columnar memory buffers`, not dynamic numeric objects like `NumPy` arrays or Pandas Series.

In [7]:
# in this cell lets see how the vectors handle the multiplication
# print(f"Here goes pyarrow: {(items * 5) for items in days}")

# Ah-Oh!! we could not do that because unlike a math engine as numpy PyArrow is a memory format. it Gives us Efficient Storage, Serialization similar to Pandas, Spark, Polars, these arrays are immutable, we cannot simply change them like that

print(f"Here goes numpy: {days_with_numpy_2 * 5}")

Here goes numpy: [ 5 10 15 20 25 30 35]


In [11]:
# Let's continue out hunt for PyArrow
days_2 = pa.array([4,6,2,7,4,9,1], type=pa.int8())
months = pa.array([1,3,5,7,9,11,13], type=pa.int8())
years = pa.array([1990, 2000, 1995, 2000, 1995, 1990, 2010], type = pa.int16())

birthdays_table = pa.table([years,days_2,months], names=['years','days','months'])

print(f" Here is how the table would look like: "
      f"{birthdays_table}")

 Here is how the table would look like: pyarrow.Table
years: int16
days: int8
months: int8
----
years: [[1990,2000,1995,2000,1995,1990,2010]]
days: [[4,6,2,7,4,9,1]]
months: [[1,3,5,7,9,11,13]]


In [20]:
# This is just for practice purposes
power_level = pa.array(['S+','A+','SS','B','C+','F'], type = pa.string())
names = pa.array(['Swap','Donald','Shreya','Ria','Henry','Derek'])
age = pa.array([26,26,21,28,31,39])

wrap_up_table = pa.table([names,age,power_level],names=['Names','Age','Power'])

print(" *** This is how the table would look like *** ")

print(wrap_up_table)

 *** This is how the table would look like *** 
pyarrow.Table
Names: string
Age: int64
Power: string
----
Names: [["Swap","Donald","Shreya","Ria","Henry","Derek"]]
Age: [[26,26,21,28,31,39]]
Power: [["S+","A+","SS","B","C+","F"]]


#### Performing `Computations`
Finally lets take a look at some of PyArrow's computational power

In [21]:
import pyarrow.compute as pc

print(pc.value_counts(birthdays_table['years']))
print("take a closer look at how the response has been for the command above")

-- is_valid: all not null
-- child 0 type: int16
  [
    1990,
    2000,
    1995,
    2010
  ]
-- child 1 type: int64
  [
    2,
    2,
    2,
    1
  ]
take a closer look at how the response has been for the command above


In [22]:
# Let's do the same for the one table that we created for practice purposes

print(pc.value_counts(wrap_up_table['Power']))

-- is_valid: all not null
-- child 0 type: string
  [
    "S+",
    "A+",
    "SS",
    "B",
    "C+",
    "F"
  ]
-- child 1 type: int64
  [
    1,
    1,
    1,
    1,
    1,
    1
  ]


#####  Let us now understand what's up with this

`Child 1`
 and
 `Child 0` thing

Let’s try to figure out what’s happening behind the scenes 👇

## 🧩 What We are Seeing is:

This output represents a **dictionary-encoded array** in `Apache Arrow`.

| Child | Description | Example Type |
|--------|--------------|---------------|
| **child 0** | Stores the **unique dictionary values** (actual strings) | `string` |
| **child 1** | Stores the **integer codes** that reference the dictionary values | `int64` |

So, in essence:

- **child 0** → `["S+", "A+", "SS", "B", "C+", "F"]`
- **child 1** → `[0, 1, 2, 3, 4, 5]`

This encoding allows Arrow to store gigantic string columns **much more efficiently** by avoiding repetition.

---

## ⚙️ Why Dictionary Encoding Exists

Dictionary encoding (also called *categorical encoding*) saves memory by:

- Storing **each `unique string` only once**.
- Replacing `repeated strings` with **integer references** (small but fixed-size).
- While keeping the mapping `consistent` and `reversible`.

This makes operations like joins, filters, and aggregations faster while reducing memory usage.

---

## 🤔 Why Does `child 1` Show All `1`s?

You might think that if we reverse the 1 to get its reference string value it might confuse Arrow  but wait:

- PyArrow **infers types automatically** and compresses them internally.
- The console output shows a **simplified or truncated** internal structure.
- It’s a *visual artifact* — the `actual mapping` in memory is still correct.

Even though `child 1` looks like `[1, 1, 1, 1, 1, 1]`, the Arrow memory engine correctly maps all values to their corresponding dictionary entries internally.

Arrow doesn’t get confused — it’s just showing a **debug view**, not the full memory layout.

---

## ✅ Verifying Actual Values

You can always inspect your data safely using `.to_pylist()`:

```python
import pyarrow as pa

arr = pa.array(["S+", "A+", "SS", "B", "C+", "F"])
print(arr.to_pylist())

In [17]:
# Let's hit it to see as well if it works or not
print(power_level.to_pylist())
# See pretty cool right ? Yes.


['S+', 'A+', 'SS', 'B', 'C+', 'F']


#####  Now we could do the `dictionary encoding` manualy

- Here's how we would do it -


In [19]:
indices = pa.array([0,1,2,3,4])
dictionary = pa.array(['Sussy','Pisi','Galium','Radium','Tortilla'])

dict_array = pa.DictionaryArray.from_arrays(indices,dictionary)

print(dict_array)
# See ?


-- dictionary:
  [
    "Sussy",
    "Pisi",
    "Galium",
    "Radium",
    "Tortilla"
  ]
-- indices:
  [
    0,
    1,
    2,
    3,
    4
  ]



##### So intersting fact - if we want to disable
##### Dictionary encoding and store plain
##### Strings instead  we can use after `pa.array([...], pa.type= string())`

##### `Insights`

- child 0 - Unique dictionary values
- child 1 -  Integer codes referencing dictionary values
- Why all 1s? - A simplified debug view - not actual data confusion
- By using type=pa.string() at the end, it basically disables encoding


##### Working with large data

`Arrow` also provides the `pyarrow.dataset` API to work with large data, which will handle for you partitioning of your data in smaller chunks

In [24]:
# We shall now try to save our table into an actual parquet file
import pyarrow.dataset as ds

ds.write_dataset(birthdays_table, "birthday_details", format='parquet',
                 partitioning=ds.partitioning(pa.schema([birthdays_table.schema.field("years")])))

print("Saving/ Updation completed")

Saving/ Updation completed


In [25]:
# Let us now load the partitioned dataset
birthdays_dataset = ds.dataset("birthday_details", format='parquet', partitioning=['years'])

print(birthdays_dataset.files)

['birthday_details/1990/part-0.parquet', 'birthday_details/1995/part-0.parquet', 'birthday_details/2000/part-0.parquet', 'birthday_details/2010/part-0.parquet']


In [27]:
#Previous out-put sort or resembles this:
#birthday_details/
#├── 1990/
#│   └── part-0.parquet
#├── 1995/
#│   └── part-0.parquet
#├── 2000/
#│   └── part-0.parquet
#└── 2010/
 #   └── part-0.

 # Each subfolder (e.g. 1990/, 1995/) corresponds to one unique value of the partition key,
# which in this case is years.

##### `How the data is Stored`
Folder ------- File ------- Data Example

`1990/` -----> `part-0.parquet` ----> rows where `years = 1990`


##### And So forth

In [28]:
# If we did something like this -
table  = birthdays_dataset.to_table()
print(table) # This gives us back everything

pyarrow.Table
days: int8
months: int8
years: int32
----
days: [[4,9],[2,4],[6,7],[1]]
months: [[1,11],[5,9],[3,7],[13]]
years: [[1990,1990],[1995,1995],[2000,2000],[2010]]


In [30]:
# Or, we could load a single partition efficiently (Arrow will read only one folder)

table_1995 = birthdays_dataset.to_table(filter = pc.field('years') == 1995)
print(table_1995) # see? this is the magic no need to worry about the whole thing, I mean we could have a data set with 10 billion rows, it would be insanely inconvenient to tackle that much at once

pyarrow.Table
days: int8
months: int8
years: int32
----
days: [[2,4]]
months: [[5,9]]
years: [[1995,1995]]
